In [28]:
import pandas as pd
import seaborn as sns
import os
import cv2
from tqdm import tqdm
import pydicom

In [2]:
covid_dataset = pd.read_csv('./covid-chestxray-dataset-master/metadata.csv')
covid_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Patientid       129 non-null    int64  
 1   offset          93 non-null     float64
 2   sex             114 non-null    object 
 3   age             110 non-null    float64
 4   finding         129 non-null    object 
 5   survival        48 non-null     object 
 6   view            129 non-null    object 
 7   date            129 non-null    object 
 8   location        97 non-null     object 
 9   filename        129 non-null    object 
 10  doi             68 non-null     object 
 11   url            129 non-null    object 
 12  license         32 non-null     object 
 13  clinical notes  124 non-null    object 
 14  other notes     60 non-null     object 
 15  Unnamed: 15     0 non-null      float64
dtypes: float64(3), int64(1), object(12)
memory usage: 16.2+ KB


In [4]:
covid_dataset.finding.value_counts()

COVID-19         104
SARS              11
Streptococcus      6
ARDS               4
Pneumocystis       2
No Finding         2
Name: finding, dtype: int64

In [6]:
covid_front = os.listdir('./covid-chestxray-dataset-master/images/')
len(covid_front)

104

In [11]:
covid_front_df = covid_dataset[(covid_dataset.filename.isin(covid_front)) & (covid_dataset.finding=='COVID-19')]
covid_front_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 0 to 126
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Patientid       76 non-null     int64  
 1   offset          56 non-null     float64
 2   sex             67 non-null     object 
 3   age             63 non-null     float64
 4   finding         76 non-null     object 
 5   survival        29 non-null     object 
 6   view            76 non-null     object 
 7   date            76 non-null     object 
 8   location        60 non-null     object 
 9   filename        76 non-null     object 
 10  doi             44 non-null     object 
 11   url            76 non-null     object 
 12  license         14 non-null     object 
 13  clinical notes  71 non-null     object 
 14  other notes     32 non-null     object 
 15  Unnamed: 15     0 non-null      float64
dtypes: float64(3), int64(1), object(12)
memory usage: 10.1+ KB


In [12]:
covid_front_df.head()

,Patientid,offset,sex,age,finding,survival,view,date,location,filename,doi,url,license,clinical notes,other notes,Unnamed: 15
0,2,0.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,PA,2020,"Changhua Christian Hospital, Changhua City, Ta...",nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [13]:
rsna_dataset = pd.read_csv('./rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')
rsna_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30227 entries, 0 to 30226
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   patientId  30227 non-null  object 
 1   x          9555 non-null   float64
 2   y          9555 non-null   float64
 3   width      9555 non-null   float64
 4   height     9555 non-null   float64
 5   Target     30227 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.4+ MB


In [14]:
rsna_dataset.Target.value_counts()

0    20672
1     9555
Name: Target, dtype: int64

In [16]:
rsna_dataset[rsna_dataset.Target==0]

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...
30216,c1cf3255-d734-4980-bfe0-967902ad7ed9,NaN,NaN,NaN,NaN,0
30217,c1e228e4-b7b4-432b-a735-36c48fdb806f,NaN,NaN,NaN,NaN,0
30218,c1e3eb82-c55a-471f-a57f-fe1a823469da,NaN,NaN,NaN,NaN,0
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0


In [25]:
for filename in tqdm(covid_front_df.filename.unique()):
    img = cv2.imread('./covid-chestxray-dataset-master/images/'+filename)
    cv2.imwrite('./images/'+filename, img)

100%|██████████| 76/76 [00:06<00:00, 11.66it/s]


In [36]:
for patientid in tqdm(rsna_dataset[rsna_dataset.Target==0].patientId.unique()[0:5]):
    ds = pydicom.dcmread('./rsna-pneumonia-detection-challenge/stage_2_train_images/'+patientid+'.dcm')
    pixel_array_numpy = ds.pixel_array
    cv2.imwrite('./images/'+patientid+'.jpeg', pixel_array_numpy)

100%|██████████| 5/5 [00:00<00:00, 45.31it/s]


In [37]:
covid_front_df.to_pickle('./covid_df.pkl')